<a href="https://colab.research.google.com/github/berryew/DREAM-PTD-Prediction-Using-Microarray/blob/master/DREAM_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages, import them, and load the dataset

In [1]:
!pip install Biopython
!pip install lifelines
!pip install imblearn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from pathlib import Path
DATA = Path("/content/drive/My Drive/E4060/DREAM/data")

In [4]:
import pandas as pd
import numpy as np

from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.utils import shuffle

import xgboost as xgb

from Bio.Cluster import kcluster

from collections import Counter

from sklearn.cluster import AgglomerativeClustering
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
# load the microarray
# Just `eset_SC2_v20.csv` and `anoSC2_v20_nokey.csv` will be sufficient. 
# `eset_SC2_v20.csv` contains information of both `HuGene21ST_RMA.csv` and `HTA20_RMA.csv`
eset = pd.read_csv(DATA / "eset_SC2_v20.csv", index_col=0)
# load the annotation
annot = pd.read_csv(DATA / "anoSC2_v20_nokey.csv")

### Define SampleID-IndividualID map, parameters and models

In [0]:
# create a SampleID-IndividualID map, so that we can group by IndividualID and calculate the average
sample_to_individual = pd.Series(annot.IndividualID.values, index=annot.SampleID).to_dict()

#### Survival model

In [0]:
# define xgboost (survival model) parameters
xgb_params = [
{
    "eta": 0.01,
    "subsample": 0.5,
    "colsample_bytree": 0.75,
    "max_depth": 5,
    "objective": "survival:cox",
    "eval_metric": "cox-nloglik",
    "seed": 4060
},
{
    "eta": 0.01,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "max_depth": 10,
    "objective": "survival:cox",
    "eval_metric": "cox-nloglik",
    "seed": 4060
}, 
{
    "eta": 0.01,
    "subsample": 0.5,
    "colsample_bytree": 0.5,
    "max_depth": 5,
    "objective": "survival:cox",
    "eval_metric": "cox-nloglik",
    "seed": 4060
}]

In [0]:
'''
`X_train`: training set input
`y_train`: training set output
`X_test`: validation input
`xgb_params`: tune parameters inside the cv of model training
output the normalized scores, as probabilities
'''
def xgb_prediction(X_train, y_train, X_test, xgb_params):
    cv_scores = []
    cv_rounds = []
    dtrain = xgb.DMatrix(X_train, label=y_train)
    for params in xgb_params:
        cvout = xgb.cv(params, dtrain, num_boost_round=500, nfold=5,
                    early_stopping_rounds=10)
        # get the best (minimum) loss score
        cv_scores.append(cvout["test-cox-nloglik-mean"].min())
        cv_rounds.append(cvout["test-cox-nloglik-mean"].idxmin())
    print(cv_scores, cv_rounds)
    xgbmodel = xgb.train(xgb_params[np.argmin(cv_scores)],
                     dtrain, num_boost_round=cv_rounds[np.argmin(cv_scores)])
    dtest = xgb.DMatrix(X_test)
    pred = xgbmodel.predict(dtest)
    # normalize the risk score, so that it can be treated as a probability
    return normalize(pred)

In [0]:
'''
`X` defines the whole dataset, train and test are the index of validation split
`method` defines clustering methods, either kmeans or hierarchical 
    * the parameters of hierarchical clustering can be set as hierarchical.euclidean.average for example
`n_cluster` defines the maximum clusters
    * based on the cllustering, leave out clusters with much less genes, and generate the features by calculate the means of each cluster
`df` decides the type of output, pandas DataFrame or numpy array
'''
def mean_by_cluster_sa(X, train, test, n_cluster=5, method='kmeans', df=True):
    
    # clustering, defined by argument `method`
    if method == 'kmeans':
        clusterid, error, nfound = kcluster(np.array(X.iloc[train].transpose()), nclusters=n_cluster)
    elif method.startswith('hierarchical'):
        _, affinity, linkage = method.split(".")
        cluster = AgglomerativeClustering(n_clusters=n_cluster, affinity=affinity, linkage=linkage)
        cluster.fit_predict(np.array(X.iloc[train].transpose()))
        clusterid = cluster.labels_

    metagenes = X.transpose().copy()
    metagenes = metagenes.reset_index(drop=True)
    metagenes['id'] = clusterid
    metagenes['size'] = metagenes.groupby('id').size()
    # candidate_condition = metagenes['size'] > 0.4 * X.shape[1]//n_cluster
    candidate_condition = metagenes['size'] > max(0.4 * X.shape[1]//n_cluster, 3)
    metagenes = metagenes.groupby('id').mean()
    # print(metagenes['size'] > 0.5 * X.shape[0]//n_cluster)
    metagenes = metagenes[candidate_condition]
    print('Before | After deleting clusters with much less objects: {} | {}'.format(Counter(clusterid), metagenes.shape[0]))
    if df:
        return metagenes.iloc[:, train].transpose(), metagenes.iloc[:, test].transpose()
    else:
        return np.array(metagenes.iloc[:, train].transpose()), np.array(metagenes.iloc[:, test].transpose())
    

In [0]:
'''
normalize the value to range from 0 to 1
'''
def normalize(array):
    return (array - min(array)) / (max(array) - min(array))

In [0]:
cidx_df = pd.read_csv(DATA / "Concordance_results_T2.csv", index_col=0)
cidx_top_genes = cidx_df.index[:100]

T2 = annot.sort_values('GA', ascending=False).drop_duplicates(['IndividualID'])
train_id_T2 = T2[T2.Train==1].SampleID
test_id_T2 = T2[T2.Train==0].SampleID

In [0]:
X_sa = eset.loc[cidx_top_genes, eset.columns.isin(train_id_T2)].transpose()
y_sa = annot.loc[annot.SampleID.isin(train_id_T2), ['Group', 'TTD']]
y_sa['Group'] = y_sa.Group != 'Control'
y_sa['Event'] = True
y_sa.index = X_sa.index
X_y_sa = pd.concat([X_sa, y_sa], axis=1)

In [0]:
seed = 4060
X_train, y_TTD, y_Group = shuffle(X_sa, y_sa.TTD.values, y_sa.Group.values, random_state=seed)
X_test = eset.loc[cidx_top_genes, eset.columns.isin(test_id_T2)].transpose()
X = pd.concat([X_train, X_test], axis=0)

In [14]:
train = range(len(X_train))
test = range(len(X_train), len(X))

X_train, X_test = mean_by_cluster_sa(X, train, test, 10, 'hierarchical.euclidean.complete')
print("True samples in training: {}".format(sum(y_Group[train])))
probas_ = xgb_prediction(X_train, y_TTD, X_test, xgb_params)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Before | After deleting clusters with much less objects: Counter({0: 31, 2: 18, 5: 13, 3: 12, 1: 9, 4: 8, 8: 6, 6: 1, 7: 1, 9: 1}) | 7
True samples in training: 64
[2.7647386000000003, 2.7554108, 2.7600056] [64, 74, 109]


##### Create a dataframe `prediction_sa`, with columns `IndividualID, sPTD, PPROM`
For survival analysis, the goal is to prediction the duration of event (delivery). Therefore, two predictions (sPTD and PPROM) are identical.

In [15]:
prediction_sa = pd.DataFrame({'SampleID': X_test.index, 'sPTD': probas_, 'PPROM': probas_})
prediction_sa['IndividualID'] = prediction_sa['SampleID'].map(sample_to_individual)
prediction_sa = prediction_sa.drop(columns='SampleID').set_index('IndividualID')
prediction_sa.head()

,sPTD,PPROM
IndividualID,,
18275,0.600750,0.600750
18427,0.238119,0.238119
18473,0.473468,0.473468
19086,0.308764,0.308764
19217,0.005074,0.005074


#### Classification

In [0]:
'''
downsampling the majority class
`X_y` defines the training matrix, row: samples, column: genes + Group
`sep` defines the index to split columns into genes + annotation, e.g. Group
`df` defines the type of output
`seed` defines the random state
`shuffle` defines if shuffuling is needed
'''
def downsampling(X_y, sep=-1, df=False, seed=47, shuffle=False):
    True_samples = X_y[X_y.Group].index

    # Shuffle the Dataset.
    X_y = X_y.sample(frac=1, random_state=7)

    # Put all the positive samples in a separate dataset.
    True_X_y = X_y.loc[X_y.index.isin(True_samples)]

    # Randomly select observations from the negative samples
    False_X_y = X_y.loc[~X_y.index.isin(True_samples)].sample(n=len(True_samples), random_state=seed)

    # Concatenate both dataframes
    downsampled_X_y = pd.concat([True_X_y, False_X_y])

    # based on the type of output: dataframe or np.array
    if df:
        X, y = downsampled_X_y.iloc[:, :sep], downsampled_X_y.iloc[:, sep:]
    else:
        X, y = np.array(downsampled_X_y.iloc[:, :sep]), np.array(downsampled_X_y.iloc[:, sep:])

    
    # based on whether the output need shuffling
    if shuffle:
        return shuffle(X, y, random_state=seed)
    else:
        return X, y

In [0]:
'''
oversampling the minority class, use SMOTE from the library `imblearn`
`X_y` defines the training matrix, row: samples, column: genes + Group
`sep` defines the index to split columns into genes + annotation, e.g. Group
`df` defines the type of output
`seed` defines the random state
`shuffle` defines if shuffuling is needed
'''
def oversampling(X_y, sep=-1, df=False, seed=47, shuffle=False):
    # Resample the minority class.
    sm = SMOTE(sampling_strategy='minority', random_state=4060)

    # Fit the model to generate the data.
    X, y = sm.fit_sample(X_y.iloc[:, :sep], X_y.iloc[:, sep:])
    
    # based on the type of output: dataframe or np.array
    if df:
        X = pd.DataFrame(X, columns=X_y.columns[:-1])
        y = pd.DataFrame(y, columns=['Group'])
    
    # based on whether the output need shuffling
    if shuffle:
        return shuffle(X, y, random_state=seed)
    else:
        return X, y

In [0]:
'''
`X_train` defines the traning set
`X_test` defines the test set
`method` defines clustering methods, either kmeans or hierarchical 
    * the parameters of hierarchical clustering can be set as hierarchical.euclidean.average for example
`n_cluster` defines the maximum clusters
    * based on the cllustering, leave out clusters with much less genes, and generate the features by calculate the means of each cluster
`df` decides the type of output, pandas DataFrame or numpy array
'''
def mean_by_cluster_clf(X_train, X_test, n_cluster=5, method='kmeans', df=True):
    
    # clustering, defined by argument `method`
    if method == 'kmeans':
        clusterid, error, nfound = kcluster(np.array(X_train.transpose()), nclusters=n_cluster)
    elif method.startswith('hierarchical'):
        _, affinity, linkage = method.split(".")
        cluster = AgglomerativeClustering(n_clusters=n_cluster, affinity=affinity, linkage=linkage)
        cluster.fit_predict(np.array(X_train.transpose()))
        clusterid = cluster.labels_

    sep = len(X_train)

    metagenes = pd.concat([X_train, X_test], axis=0, sort=False).transpose().copy()
    metagenes = metagenes.reset_index(drop=True)
    metagenes['id'] = clusterid
    metagenes['size'] = metagenes.groupby('id').size()
    # candidate_condition = metagenes['size'] > 0.4 * X.shape[1]//n_cluster
    candidate_condition = metagenes['size'] > max(0.4 * X.shape[1]//n_cluster, 3)
    # candidate_condition = metagenes['size'] > 4
    # print(metagenes.head(1))
    metagenes = metagenes.groupby('id').mean()
    # print(metagenes['size'] > 0.5 * X.shape[0]//n_cluster)
    metagenes = metagenes[candidate_condition].drop(columns='size')
    print('Before | After deleting clusters with much less objects: {} | {}'.format(Counter(clusterid), metagenes.shape[0]))
    
    if df:
        return metagenes.iloc[:, :sep].transpose(), metagenes.iloc[:, sep:].transpose()
    else:
        return np.array(metagenes.iloc[:, :sep].transpose()), np.array(metagenes.iloc[:, sep:].transpose())
    

In [0]:
train_id = annot[annot.Train == 1]['SampleID']
test_id = annot[annot.Train == 0]['SampleID']

In [20]:
categories = ['sPTD', 'PPROM']

prediction_clf = pd.DataFrame({'SampleID': test_id})
prediction_clf['IndividualID'] = prediction_clf['SampleID'].map(sample_to_individual)

for category in categories:
    file_name = "topTable1000_" + category + ".csv"
    topTable = pd.read_csv(DATA / file_name, index_col=0)
    top_genes = topTable.index[:100]

    X_clf = eset.loc[top_genes, eset.columns.isin(train_id)].transpose()
    y_clf = annot.loc[annot.SampleID.isin(train_id), ['Group']]
    y_clf['Group'] = y_clf['Group'] == category
    X_y_clf = pd.concat([X_clf.reset_index(), y_clf.reset_index(drop=True)], axis=1)
    X_y_clf = X_y_clf.set_index('index')

    seed = 4060
    random_state = np.random.RandomState(seed//2)
    X_train, y_train = oversampling(X_y_clf, df=True)
    X_train, y_train = shuffle(X_train, y_train, random_state=seed)
    y_train = np.ravel(y_train)
    X_test = eset.loc[top_genes, eset.columns.isin(test_id)].transpose()
    
    X = pd.concat([X_train, X_test], axis=0)
    train = range(len(X_train))
    test = range(len(X_train), len(X))

    classifier = svm.SVC(kernel='linear', probability=True, random_state=random_state)
    
    X_train, X_test = mean_by_cluster_sa(X, train, test, 10, 'hierarchical.euclidean.complete', df=False)
    print("True samples in training: {}".format(sum(y_train)))
    probas_ = classifier.fit(X_train, y_train).predict_proba(X_test)
    prediction_clf[category] = probas_[:, 1]

Before | After deleting clusters with much less objects: Counter({9: 18, 3: 18, 6: 18, 2: 13, 1: 10, 7: 9, 0: 8, 5: 3, 4: 2, 8: 1}) | 7
True samples in training: 380
Before | After deleting clusters with much less objects: Counter({2: 23, 6: 20, 1: 19, 3: 16, 4: 7, 0: 7, 5: 4, 7: 2, 8: 1, 9: 1}) | 6
True samples in training: 340


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


##### Create a dataframe prediction_clf, with columns `IndividualID, sPTD, PPROM`

In [21]:
prediction_clf = prediction_clf.groupby('IndividualID').mean()
prediction_clf.head()

,sPTD,PPROM
IndividualID,,
18223,0.726774,0.620952
18275,0.084437,0.414845
18427,0.053158,0.535086
18453,0.192672,0.398344
18473,0.115525,0.413953


#### Combine two models, and obtain the averages respectively 

In [22]:
prediction_combine = pd.concat([prediction_sa, prediction_clf])
prediction_combine = prediction_combine.groupby(prediction_combine.index).mean()
prediction_combine.head()

,sPTD,PPROM
IndividualID,,
18223,0.455851,0.402940
18275,0.342594,0.507797
18427,0.145639,0.386603
18453,0.211595,0.314430
18473,0.294497,0.443711
